In [1]:
import pandas as pd
from ortools.sat.python import cp_model

data = pd.read_csv('TRIAL_DATA.csv')
new_dataset = data.dropna().copy()
print('Dataset: ', new_dataset)

# Priority mapping
priority_mapping = {
    "Locked In activity": 1,
    "Scheduled activity": 2,
    "Recurring high activity": 3,
    "Recurring medium activity": 4,
    "Recurring low activity": 5,
    "To-do activity": 6,
    "Bucket list activity": 7,
}

# Map categories to priorities
new_dataset['priority'] = new_dataset['category'].map(priority_mapping)
new_dataset.drop(columns=['category'], inplace=True)

# Convert times to minutes
def time_to_minutes(time_string):
    if pd.isnull(time_string):
        return None
    time = pd.to_datetime(time_string, format= '%H:%M').time()
    return time.hour * 60 + time.minute
    
new_dataset['preferred_time_start'] = new_dataset['preferred_time_start'].apply(time_to_minutes)
new_dataset['preferred_time_end'] = new_dataset['preferred_time_end'].apply(time_to_minutes)

# Sort dataset by priority
new_dataset.sort_values(by='priority', inplace=True)

# # Sort by date first
# new_dataset_sorted_date = new_dataset.sort_values(by='start_date', inplace=False)

# # Then sort by preferred time start within the sorted data
# new_dataset_sorted_date_time = new_dataset_sorted_date.sort_values(by='preferred_time_start', inplace=False)

# print('Sorted Date: ', new_dataset_sorted_date)
# print('Sorted Date and Time: ', new_dataset_sorted_date_time)


model = cp_model.CpModel()

start_vars = {}
end_vars = {}
intervals = []
new_dataset['estimated_durations'] = new_dataset['estimated_durations'].fillna(60)
durations = new_dataset['estimated_durations'].astype(int)

max_days = 7
day_vars = {}

def calculate_penalty_with_priority(start_time, preferred_start, priority):
    deviation_penalty = model.NewIntVar(0, 1440, f"penalty_{i}")
    model.AddAbsEquality(deviation_penalty, start_time - preferred_start)
    weighted_penalty = model.NewIntVar(0, 1440, f"weighted_penalty_{i}")
    model.Add(weighted_penalty == deviation_penalty * (7 - priority))
    return weighted_penalty

for i, row in new_dataset.iterrows():
    preferred_start = int(row['preferred_time_start'])
    preferred_end = int(row['preferred_time_end'])
    duration = int(durations[i])

    start_vars[i] = model.NewIntVar(preferred_start, preferred_end - duration, f"start_{i}")
    end_vars[i] = model.NewIntVar(preferred_start + duration, preferred_end, f"end_{i}")
    day_vars[i] = model.NewIntVar(0, max_days - 1, f"day_{i}")

    model.Add(start_vars[i] >= day_vars[i] * 1440)
    model.Add(start_vars[i] < (day_vars[i] + 1) * 1440)

    model.Add(end_vars[i] == start_vars[i] + duration)

    deviation_penalty = calculate_penalty_with_priority(start_vars[i], preferred_start, row['priority'])

    interval = model.NewIntervalVar(start_vars[i], duration, end_vars[i], f"interval_{i}")
    intervals.append(interval)

model.AddNoOverlap(intervals)

removed_indices = []

for i in range(len(new_dataset) - 1):
    row_up = new_dataset.iloc[i]
    row_down = new_dataset.iloc[i + 1]
    
    if not row_up.equals(row_down):  # Check if rows are different
        # Check if activities overlap
        if row_up['preferred_time_start'] < row_down['preferred_time_end'] and row_down['preferred_time_start'] < row_up['preferred_time_end']:
            print(f"Conflict Found between activity {i} and activity {i+1}")

            # Prioritize removing lower-priority activity
            if row_up['priority'] < row_down['priority']:
                print(f"Removing activity {i} (lower priority)")
                removed_indices.append(i)  # Mark the lower-priority activity for removal
            else:
                print(f"Removing activity {i+1} (lower priority)")
                removed_indices.append(i + 1)  # Mark the lower-priority activity for removal

# Remove the activities with lower priority
new_dataset = new_dataset.drop(removed_indices).reset_index(drop=True)
print(f"Activities removed: {removed_indices}")
print("Updated dataset without removed activities:")
print("====================================")

print(new_dataset)

# # Rebuild model after removing the activity
# model = cp_model.CpModel()
# start_vars = {}
# end_vars = {}
# intervals = []
# durations = new_dataset['estimated_durations'].astype(int)

# # Rebuild the model with the updated dataset
# for i, row in new_dataset.iterrows():
#     preferred_start = int(row['preferred_time_start'])
#     preferred_end = int(row['preferred_time_end'])
#     duration = int(durations[i])

#     start_vars[i] = model.NewIntVar(preferred_start, preferred_end - duration, f"start_{i}")
#     end_vars[i] = model.NewIntVar(preferred_start + duration, preferred_end, f"end_{i}")
#     day_vars[i] = model.NewIntVar(0, max_days - 1, f"day_{i}")

#     model.Add(start_vars[i] >= day_vars[i] * 1440)
#     model.Add(start_vars[i] < (day_vars[i] + 1) * 1440)

#     model.Add(end_vars[i] == start_vars[i] + duration)

#     deviation_penalty = calculate_penalty_with_priority(start_vars[i], preferred_start, row['priority'])

#     interval = model.NewIntervalVar(start_vars[i], duration, end_vars[i], f"interval_{i}")
#     intervals.append(interval)

# model.AddNoOverlap(intervals)

# # Solve the model
# solver = cp_model.CpSolver()
# solver.parameters.max_time_in_seconds = 60

# status = solver.solve(model)
# print('Status: ', status)
# if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
#     print("Schedule: ")
#     for i in range(len(new_dataset)):
#         start_time = solver.Value(start_vars[i])
#         start_hour, start_minute = divmod(start_time, 60)
#         end_time = solver.Value(end_vars[i])
#         end_hour, end_minute = divmod(end_time, 60)

#         day = solver.Value(day_vars[i])
#         day_start_time = 1440 * day
#         adjusted_start_time = start_time + day_start_time
#         adjusted_end_time = end_time + day_start_time
#         print(f"Activity {new_dataset.iloc[i]['activity_name']} starts at {adjusted_start_time // 60:02}:{adjusted_start_time % 60:02} and ends at {adjusted_end_time // 60:02}:{adjusted_end_time % 60:02} on Day {day + 1}")
# else:
#     print("No feasible schedule found")

Dataset:     activity_name            category involved_person  estimated_durations  \
0     activity_1  Locked In activity             Sam                 59.0   
1     activity_2  Locked In activity             Sam                 59.0   
2     activity_3  Locked In activity             Sam                 59.0   
3     activity_4  Locked In activity             Sam                 59.0   
4     activity_5  Locked In activity             Sam                 59.0   
5     activity_6  Locked In activity             Sam                 59.0   
6     activity_7  Locked In activity             Sam                 59.0   
7     activity_8  Locked In activity             Sam                 59.0   
8     activity_9  Locked In activity             Sam                 59.0   
9    activity_10  Locked In activity             Sam                 59.0   
10   activity_11  Locked In activity             Sam                 59.0   
11   activity_12  Locked In activity             Sam              

C:\Users\Brend\Downloads\AI-Personal-Assistant-Scheduler
